# LOGISTIC REGRESSION

### importing libraries

In [209]:
#importing libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
import math

In [210]:

from sklearn.linear_model import Ridge
from sklearn.metrics import f1_score as f1
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression as logreg
from sklearn.linear_model import LinearRegression as linreg
from sklearn.metrics import mean_squared_error as rmse

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score ,roc_auc_score as auc, accuracy_score as acc, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve,average_precision_score,average_precision_score as aps,log_loss as ll

from pandas.core.common import SettingWithCopyWarning
from sklearn.exceptions import FitFailedWarning
import warnings
warnings.simplefilter(action='ignore', category=FitFailedWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [211]:
from sklearn.metrics import make_scorer

# random state

In [212]:
r=6

### importing data

In [213]:
sklearn.__version__

'1.0.1'

In [214]:
data = pd.read_csv('FILTERED[csv].csv')
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [215]:
data=(data[[z for z in data.columns.tolist() if z not in ['Unnamed: 0']]]).copy()

In [216]:
target='stroke'   
makescorer=make_scorer(auc, greater_is_better=True, needs_proba=True, needs_threshold=False)
model= logreg()
lr=linreg()
scaler = MinMaxScaler()

In [217]:
data.shape

(5110, 11)

In [218]:
 x = (data[[z for z in data.columns.tolist() if z not in [target]]]).copy()
 y =data[target].copy() 
 x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns.tolist()).copy()

 train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
 train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

 model.fit(train_x,train_y)
 pred=model.predict_proba(val_x)[:,1]

In [219]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [220]:
kval_n=2
def kval(main,step,n=kval_n):

 klist=[]
 if (step==0):
    step=0.0001
 i=main-step*n
 if (main>=0):
  while (i<=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
 else:
  while (i>=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
  

## N FOLD CROSS VALIDATION

### to change eval metric throughout the code make corresponding changes in the eval_metric and cv_score() and eval_score()

In [221]:

nfold=9
eval_metric=['AUC']

the function takes the ml algorithm,columns of datatable and the break value as parameter . function randomly chooses nfold sets of train and validation sets where in each fold val set is complement of train set and the union of validation sets in all folds gives the dataset itself . each fold is trained on train set and tested on validation set and gives a set of evaluation metric scores

In [222]:
evallist=[]
def cv_score(ml_model=model, rstate = r, cols = x.columns.tolist()):
    i = 1
    cv_scores = []
    df1 = data.copy()
    df1 = data[cols]
    f1list=[]
    indexlist=[]
    
    # 5 Fold cross validation stratified on the basis of target
    kf = StratifiedKFold(n_splits=nfold,random_state=rstate,shuffle=True)
    for df_index,test_index in kf.split(df1,y):
        
        xtr,xvl = [df1.loc[df_index],df1.loc[test_index]]
        ytr,yvl = [y.loc[df_index],y.loc[test_index]]
        
        xtr =pd.DataFrame(scaler.fit_transform(xtr),columns=cols)
        xvl = pd.DataFrame(scaler.fit_transform(xvl),columns=cols)
            
        # Define model for fitting on the training set for each fold
        model = ml_model
        model.fit(xtr, ytr)
        pred_probs = model.predict_proba(xvl)

        evalscore=auc(yvl,pred_probs[:,1])
        evallist.append(evalscore)
        
         # Save scores
        indexlist.append(("kfold "+str((i))))
        i=i+1
        
    cv_score=pd.DataFrame(evallist,columns=eval_metric,index=indexlist)
    return cv_score

In [223]:
cv_score()

,AUC
kfold 1,0.848082
kfold 2,0.862434
kfold 3,0.849735
kfold 4,0.820767
kfold 5,0.835582
kfold 6,0.868585
kfold 7,0.828301
kfold 8,0.803909
kfold 9,0.852263


In [224]:
     def eval_score(ml_model=model, rstate = r, cols =x.columns.tolist()):
        i = 1
        indexlist=[]
    
        # Define model for fitting on the training set for each fold
        model = ml_model
        model.fit(train_x[cols],train_y)
        pred = model.predict_proba(val_x[cols])
         
        evalscore=auc(val_y,pred[:,1])
        return evalscore

In [225]:
eval_score()

0.8305390492359932

## rfe

this is a ranking of features based on how significant they are to our analysis according to RFE

In [226]:
from sklearn.feature_selection import RFE

# Create the RFE object and rank each feature

rfe = RFE(estimator=model, n_features_to_select=1, step=1)
rfe.fit(train_x, train_y)
ranking_df = pd.DataFrame()
ranking_df['Feature_name'] = x.iloc[0,].index
ranking_df['Rank'] = rfe.ranking_
ranked = ranking_df.sort_values(by=['Rank']).copy()

In [227]:
ranked

,Feature_name,Rank
1,age,1
7,avg_glucose_level,2
3,heart_disease,3
4,ever_married,4
2,hypertension,5
5,work_type,6
9,smoking_status,7
6,Residence_type,8
0,gender,9
8,bmi,10


## clustering

In [228]:
from sklearn.cluster import AgglomerativeClustering

In [229]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [230]:
ac = AgglomerativeClustering(n_clusters=5)
clusters = ac.fit(pd.DataFrame(data_scaled))

In [231]:
clusters.labels_[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [232]:
pd.Series(clusters.labels_).value_counts()

2    2606
0    1645
3     381
1     249
4     229
dtype: int64

In [233]:
clusters.labels_

array([1, 1, 1, ..., 2, 2, 2])

In [234]:
data['cluster']=clusters.labels_

In [235]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,cluster
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1,1


In [236]:
cluster_targetmean=[]
for i in data['cluster'].value_counts().index.tolist():
    cluster_targetmean.append(data[data['cluster']==i][target].mean())

In [237]:
clusterlabel=data['cluster'].value_counts().index.tolist()
clusterval=data['cluster'].value_counts().values.tolist()

In [238]:
cluster_target=pd.DataFrame([clusterlabel,clusterval,cluster_targetmean],index=['clusterlabel','label freq in data','cluster_targetmean']).transpose().copy()
cluster_target

,clusterlabel,label freq in data,cluster_targetmean
0,2.0,2606.0,0.0
1,0.0,1645.0,0.0
2,3.0,381.0,0.0
3,1.0,249.0,1.0
4,4.0,229.0,0.0


In [239]:
data=(data[[z for z in data.columns.tolist() if z not in ['cluster']]]).copy()

In [240]:
x = (data[[z for z in data.columns.tolist() if z not in [target]]]).copy()
y = data[target].copy()
x.shape, y.shape

train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r).copy()
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r).copy()

## feature binning

## decision tree for continous columns

In [241]:
data.dtypes

gender                 int64
age                  float64
hypertension           int64
heart_disease          int64
ever_married           int64
work_type              int64
Residence_type         int64
avg_glucose_level    float64
bmi                  float64
smoking_status         int64
stroke                 int64
dtype: object

In [242]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [243]:
contlist=[]
for i in data[[x for x in data.columns.tolist() if x not in target]]:
 if ((type(data[i][5])==np.float64)|len(data[i].value_counts())>6):
    contlist.append(i)

In [244]:
contlist

['age', 'avg_glucose_level', 'bmi']

In [245]:
catlist=[t for t in x.columns.tolist() if t not in contlist]
catlist

['gender',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'smoking_status']

In [246]:
from sklearn.tree import DecisionTreeClassifier as dtc,export_graphviz
from sklearn.model_selection import GridSearchCV as gs,RandomizedSearchCV as rs
from feature_engine.discretisation import DecisionTreeDiscretiser as dtd
import graphviz

In [247]:
depth=kval(15,1,15)
decimal=kval(0.035,(0.035/4),4)
subdecimal=kval(0.01,(0.01/4),4)

In [248]:
dt=dtc(random_state=r)

In [249]:
for k in contlist:
 val_x1=val_x[k].to_frame()
 train_x1=train_x[k].to_frame()  

 gsdict={'criterion':['gini','entropy'],
  'max_depth':depth,
  'min_samples_split':subdecimal,
  'min_weight_fraction_leaf':subdecimal}

 gsbest=gsdict.copy()

 val_x1=val_x[k].to_frame().copy()
 train_x1=train_x[k].to_frame().copy()  
 
 for j in gsdict :
  tree_para = {j:gsdict[j]}
  dtmodel = rs(dt, tree_para)
  dtmodel.fit(train_x1,train_y)
  gsbest[j]=[dtmodel.best_estimator_.get_params()[j]]
    
 gsbest_indi=gsbest.copy()
    
 gsdict={'criterion':['gini','entropy'],
  'max_depth':(gsbest['max_depth'][0],1,5),
  'min_samples_split':(gsbest['min_samples_split'][0],gsbest['min_samples_split'][0]/20,4),
  'min_weight_fraction_leaf':(gsbest['min_weight_fraction_leaf'][0],gsbest['min_weight_fraction_leaf'][0]/20,4)}

 for j in gsbest_indi:
  gsbest_indi[j]=gsdict[j]
  dtmodel = rs(dt, gsbest_indi)
  dtmodel.fit(train_x1,train_y)
  gsbest_indi[j]=gsbest[j]
  gsbest[j]=[dtmodel.best_estimator_.get_params()[j]]

 dtdmodel=dtd(variables=k,
    param_grid=gsbest,
    regression =False,
    random_state=r)
 dtdmodel.fit(train_x1,train_y)
 x[str(k)+"-Tree"]=dtdmodel.transform(pd.DataFrame(x[k])).copy()

In [250]:
x.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,age-Tree,avg_glucose_level-Tree,bmi-Tree
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,0.166667,0.145749,0.043137
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,0.083333,0.145749,0.077174
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,0.275862,0.029611,0.043137
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,0.048780,0.000000,0.043137
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,0.145833,0.000000,0.030111


## categorical feature frequency creation

In [251]:
catlist

['gender',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'smoking_status']

In [252]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [253]:
for i in catlist:
    x[str(i)+"_meantarget"] =data[i].map(dict(data.groupby(i)[target].mean()))

In [254]:
x.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,age-Tree,avg_glucose_level-Tree,bmi-Tree,gender_meantarget,hypertension_meantarget,heart_disease_meantarget,ever_married_meantarget,work_type_meantarget,Residence_type_meantarget,smoking_status_meantarget
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,0.166667,0.145749,0.043137,0.051064,0.039679,0.170290,0.065613,0.050940,0.052003,0.079096
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,0.083333,0.145749,0.077174,0.047094,0.039679,0.041787,0.065613,0.079365,0.045346,0.047569
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,0.275862,0.029611,0.043137,0.051064,0.039679,0.170290,0.065613,0.050940,0.045346,0.047569
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,0.048780,0.000000,0.043137,0.047094,0.039679,0.041787,0.065613,0.050940,0.052003,0.053232
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,0.145833,0.000000,0.030111,0.047094,0.132530,0.041787,0.065613,0.079365,0.045346,0.047569


In [255]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

In [256]:
finalcols=x.columns.tolist()

## backward feature elimination

1 column is removed whose removal causes least decrease in eval metric out of the present columns , this process continues till n columns are removed

In [257]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import make_scorer
def backfeat(nfeat):
 sfs1 = sfs(model, k_features=nfeat, forward=False)
 sfs0 = sfs1.fit(train_x, train_y)
 featlist = list(sfs0.k_feature_names_)
 featlist
 
 return featlist

In [258]:
#LARGE EXECUTION TIME ESPECIALLY TO REDUCE TO SMALLER NO OF COLUMNS
#increase step to to even execute it on next block 
backcollist,backfeatlist=[[],[]]
step=1
for i in range(4,(len(x.columns.tolist())),step):
   backfeatlist.append(backfeat(i))
   backcollist.append(i)

In [259]:
backfeatlist

[['gender', 'age', 'hypertension', 'heart_disease'],
 ['gender', 'age', 'hypertension', 'heart_disease', 'work_type'],
 ['gender',
  'age',
  'hypertension',
  'heart_disease',
  'work_type',
  'Residence_type'],
 ['gender',
  'age',
  'hypertension',
  'heart_disease',
  'work_type',
  'Residence_type',
  'avg_glucose_level'],
 ['gender',
  'age',
  'hypertension',
  'heart_disease',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'hypertension_meantarget'],
 ['gender',
  'age',
  'hypertension',
  'heart_disease',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'hypertension_meantarget',
  'work_type_meantarget'],
 ['gender',
  'age',
  'hypertension',
  'heart_disease',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'hypertension_meantarget',
  'work_type_meantarget'],
 ['gender',
  'age',
  'hypertension',
  'heart_disease',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status',
  'hypertension_meantarget

In [260]:
 scorelist=[]
 topfeatlist=[]
 topfeatscorelist=[]
 score=0
 max_=0
 for j in backcollist:
  x1=x[backfeatlist[backcollist.index(j)]].copy()
  trainx,testx,trainy,testy = train_test_split(x1,y,test_size=1/4,random_state= r,stratify=y).copy()
  trainx,valx,trainy,valy= train_test_split(trainx, trainy, test_size=1/3,random_state=r,stratify=trainy).copy()
    
  model.fit(trainx,trainy)
  pred= model.predict_proba(valx).copy()
  score=auc(valy,pred[:,1])
  scorelist.append(score)
  if(score>max_):
    max_=score
    topn_feat=j
 topfeatlist.append(topn_feat)
 topfeatscorelist.append(max_)  

In [261]:
scorelist

[0.8423307512733447,
 0.8443070882852292,
 0.8415282788624787,
 0.8385903013582343,
 0.8385637733446519,
 0.8382321731748728,
 0.8399034380305603,
 0.8294116086587436,
 0.8378077249575552,
 0.8370782045840408,
 0.8317195458404074,
 0.8352477716468591,
 0.8304727292020373,
 0.8351681876061121,
 0.8348100594227503,
 0.840964558573854]

In [262]:
#LARGE EXECUTION TIME ESPECIALLY TO REDUCE TO SMALLER NO OF COLUMNS
backcollist,backfeatlist=[[],[]]
if (topn_feat+step)>len(x.columns):
 upperlimit=len(x.columns)
else:
 upperlimit=(topn_feat+step)
for i in range((topn_feat-step+1),upperlimit):
    backfeatlist.append(backfeat(i))
    backcollist.append(i)

In [263]:

 topfeatlist=[]
 topfeatscorelist=[]
 score=0
 max_=0
 for j in backcollist:
  x1=x[backfeatlist[backcollist.index(j)]].copy()
  trainx,testx,trainy,testy = train_test_split(x1,y,test_size=1/4,random_state= r,stratify=y).copy()
  trainx,valx,trainy,valy= train_test_split(trainx, trainy, test_size=1/3,random_state=r,stratify=trainy).copy()
 
  model.fit(trainx,trainy)
  pred= model.predict_proba(valx).copy()
  score=auc(valy,pred[:,1])

  if(score>max_):
    max_=score
    topn_feat_back=j
 topfeatlist.append(topn_feat_back)
 topfeatscorelist.append(max_)  

In [265]:
topfeatlist,topfeatscorelist

([5], [0.8443070882852292])

In [266]:
backfeatcol=(topfeatlist[0])
backfeatcol

5

In [267]:
backfeatcols=backfeatlist[backcollist.index(topn_feat_back)]
backfeatcols

['gender', 'age', 'hypertension', 'heart_disease', 'work_type']

In [268]:
x=x[backfeatcols].copy()
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

# test

## with main columns with respect to each evaluation metric

In [269]:
 predicted_wrt_evaluationmetrics=[]
 model.fit(train_x,train_y)
 test_predict= model.predict_proba(test_x).copy()
 eval_score1=auc(test_y,test_predict[:,1])
 predicted_wrt_evaluationmetrics.append(test_predict[:,1])

In [270]:
eval_score1

0.8409579265704584

In [271]:
test_predict[:,1][:100]

array([0.05878971, 0.01232124, 0.16055998, 0.05207465, 0.07403989,
       0.03061624, 0.00544451, 0.01397726, 0.00456669, 0.00272958,
       0.0558737 , 0.01343622, 0.01952264, 0.02358081, 0.0340648 ,
       0.00456669, 0.21223458, 0.01474087, 0.05027586, 0.06581479,
       0.0010745 , 0.06481936, 0.1468511 , 0.04392419, 0.00763973,
       0.15603103, 0.03829898, 0.0060684 , 0.02664783, 0.01418722,
       0.00879273, 0.00630724, 0.01474087, 0.0212999 , 0.08572288,
       0.05761908, 0.00368882, 0.00805993, 0.04482982, 0.00113401,
       0.21572347, 0.00432781, 0.00368882, 0.243657  , 0.0083765 ,
       0.05222181, 0.09461786, 0.02083509, 0.01474087, 0.00070414,
       0.17386478, 0.24975441, 0.01014169, 0.04546399, 0.021965  ,
       0.03819685, 0.11818369, 0.17314046, 0.07524777, 0.0012179 ,
       0.0235536 , 0.00080759, 0.0558737 , 0.01879283, 0.0832916 ,
       0.12001839, 0.0015352 , 0.01573701, 0.00244811, 0.00865176,
       0.08891238, 0.00227967, 0.15858711, 0.00750829, 0.00616

In [272]:
np.array(test_y)[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [273]:
x[target]=data[target]

In [274]:
x.head()

,gender,age,hypertension,heart_disease,work_type,stroke
0,1,67.0,0,1,2,1
1,0,61.0,0,0,3,1
2,1,80.0,0,1,2,1
3,0,49.0,0,0,2,1
4,0,79.0,1,0,3,1


In [275]:
x.to_csv('COLUMN FILTERED[lmain].csv')